In [ ]:
import pandas as pd
import random
from faker import Faker
from datetime import date, timedelta

fake = Faker('en_IN')   # en_IN gives Indian names and cities
random.seed(42)         # fixed seed so data is the same every run
Faker.seed(42)


In [ ]:

segments          = ['Premium', 'Standard', 'Basic']
segment_weights   = [0.20, 0.50, 0.30]

channel_ids       = [1, 2, 3, 4, 5, 6, 7, 8]
channel_weights   = [0.25, 0.20, 0.10, 0.15, 0.10, 0.08, 0.07, 0.05]

cities = [
    'Mumbai', 'Delhi', 'Bangalore', 'Hyderabad', 'Chennai',
    'Pune', 'Kolkata', 'Ahmedabad', 'Jaipur', 'Surat'
]

customers = []

for i in range(1, 2001):

    segment     = random.choices(segments, weights=segment_weights)[0]
    join_date   = fake.date_between(start_date=date(2020, 1, 1), end_date=date(2024, 6, 30))

    # income depends on segment
    if segment == 'Premium':
        income = round(random.uniform(1500000, 5000000), 2)   # 15L to 50L
    elif segment == 'Standard':
        income = round(random.uniform(400000, 1500000), 2)    # 4L to 15L
    else:
        income = round(random.uniform(100000, 400000), 2)     # 1L to 4L


    is_active  = 1 if random.random() > 0.08 else 0


    nps = random.randint(0, 10)

    customers.append({
        'full_name'              : fake.name(),
        'email'                  : f'user{i}@finsightbank.com',   # unique emails
        'city'                   : random.choice(cities),
        'segment'                : segment,
        'annual_income'          : income,
        'acquisition_channel_id' : random.choices(channel_ids, weights=channel_weights)[0],
        'join_date'              : join_date,
        'is_active'              : is_active,
        'nps_score'              : nps
    })

df_customers = pd.DataFrame(customers)

print(f'Customers generated: {len(df_customers)}')
print(df_customers.head(3))

Customers generated: 2000
    full_name                   email       city   segment  annual_income  \
0  Anvi Konda  user1@finsightbank.com  Bangalore  Standard      427511.83   
1  Gagan Sami  user2@finsightbank.com     Mumbai  Standard     1381397.52   
2  Tejas Kaul  user3@finsightbank.com     Jaipur  Standard      955890.82   

   acquisition_channel_id   join_date  is_active  nps_score  
0                       5  2022-11-15          1          3  
1                       1  2021-01-01          1          6  
2                       2  2020-02-22          0          3  


In [ ]:
# Check segment distribution — should be roughly 20/50/30
print('Segment breakdown:')
print(df_customers['segment'].value_counts())

print('\nActive vs Churned:')
print(df_customers['is_active'].value_counts())

print('\nTop cities:')
print(df_customers['city'].value_counts())

Segment breakdown:
segment
Standard    1059
Basic        569
Premium      372
Name: count, dtype: int64

Active vs Churned:
is_active
1    1845
0     155
Name: count, dtype: int64

Top cities:
city
Kolkata      230
Delhi        216
Surat        204
Bangalore    198
Ahmedabad    198
Chennai      194
Jaipur       192
Mumbai       191
Pune         189
Hyderabad    188
Name: count, dtype: int64


In [ ]:
# Each customer gets between 5 and 20 transactions
# Totals around 20,000 rows

transaction_types  = ['Deposit', 'Withdrawal', 'Fee', 'Interest', 'Payment']
type_weights       = [0.35, 0.30, 0.15, 0.10, 0.10]

# product_ids 1 to 11 (matches what we inserted in Phase 1)
product_ids = list(range(1, 12))

transactions = []

for cust_index, customer in df_customers.iterrows():

    customer_id = cust_index + 1   # IDs start from 1
    join_date   = customer['join_date']
    num_txns    = random.randint(5, 20)

    for _ in range(num_txns):

        txn_type  = random.choices(transaction_types, weights=type_weights)[0]
        txn_date  = fake.date_between(start_date=join_date, end_date=date(2025, 6, 30))
        date_id   = int(txn_date.strftime('%Y%m%d'))

        # transaction amount based on type
        if txn_type == 'Deposit':
            amount  = round(random.uniform(500, 50000), 2)
            revenue = round(amount * 0.001, 2)   # bank earns 0.1% on deposits

        elif txn_type == 'Withdrawal':
            amount  = round(random.uniform(200, 20000), 2)
            revenue = 0.00

        elif txn_type == 'Fee':
            amount  = round(random.uniform(50, 500), 2)
            revenue = amount   # 100% of fees is revenue

        elif txn_type == 'Interest':
            amount  = round(random.uniform(100, 5000), 2)
            revenue = round(amount * 0.80, 2)   # bank keeps 80% of interest spread

        else:   # Payment
            amount  = round(random.uniform(100, 10000), 2)
            revenue = round(amount * 0.015, 2)  # 1.5% payment processing fee

        # 3% of transactions fail
        is_successful = 1 if random.random() > 0.03 else 0

        transactions.append({
            'customer_id'      : customer_id,
            'product_id'       : random.choice(product_ids),
            'date_id'          : date_id,
            'transaction_type' : txn_type,
            'amount'           : amount,
            'revenue_amount'   : revenue if is_successful else 0.00,
            'is_successful'    : is_successful
        })

df_transactions = pd.DataFrame(transactions)

print(f'Transactions generated: {len(df_transactions)}')
print(df_transactions.head(3))

Transactions generated: 24868
   customer_id  product_id   date_id transaction_type    amount  \
0            1           6  20250417          Deposit   6516.80   
1            1           7  20240609          Deposit  30166.75   
2            1           9  20250509          Deposit  15159.15   

   revenue_amount  is_successful  
0            6.52              1  
1           30.17              1  
2           15.16              1  


In [ ]:
print('Transaction type breakdown:')
print(df_transactions['transaction_type'].value_counts())

print(f'\nTotal revenue generated: Rs. {df_transactions["revenue_amount"].sum():,.2f}')
print(f'Average transaction amount: Rs. {df_transactions["amount"].mean():,.2f}')
print(f'Failed transactions: {df_transactions["is_successful"].value_counts().get(0, 0)}')

Transaction type breakdown:
transaction_type
Deposit       8760
Withdrawal    7456
Fee           3726
Interest      2477
Payment       2449
Name: count, dtype: int64

Total revenue generated: Rs. 6,236,020.78
Average transaction amount: Rs. 12,774.55
Failed transactions: 786


In [ ]:
df_customers.to_csv('customers.csv', index=False)
df_transactions.to_csv('transactions.csv', index=False)

print('Saved: customers.csv')
print('Saved: transactions.csv')
print(f'  customers    : {len(df_customers)} rows')
print(f'  transactions : {len(df_transactions)} rows')

Saved: customers.csv
Saved: transactions.csv
  customers    : 2000 rows
  transactions : 24868 rows
